In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as LightGBM
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

random.seed(999)
np.random.seed(999)

In [2]:
train, test = pd.read_csv('train.csv'), pd.read_csv('test.csv')
train.shape, test.shape

/home/kgautam/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


((51490, 42), (77235, 42))

In [3]:
train.isna().sum()

Report Number                         0
Local Case Number                  6453
Agency Name                           0
ACRS Report Type                      0
Crash Date/Time                       0
Route Type                         4968
Road Name                          4502
Cross-Street Type                  5019
Cross-Street Name                  4522
Off-Road Description              44540
Municipality                      43923
Related Non-Motorist              47842
Collision Type                      299
Weather                            4029
Surface Condition                  5879
Light                               704
Traffic Control                    7774
Driver Substance Abuse             9095
Non-Motorist Substance Abuse      48106
Person ID                             0
Injury Severity                       0
Circumstance                      39738
Drivers License State              2528
Vehicle ID                            0
Vehicle Damage Extent               397


In [4]:
test.isna().sum()

Id                                    0
Report Number                         0
Local Case Number                 10073
Agency Name                           0
ACRS Report Type                      0
Crash Date/Time                       0
Route Type                         7467
Road Name                          7069
Cross-Street Type                  7480
Cross-Street Name                  7071
Off-Road Description              70166
Municipality                      68737
Related Non-Motorist              74987
Collision Type                      245
Weather                            5996
Surface Condition                  9053
Light                               620
Traffic Control                   12050
Driver Substance Abuse            14083
Non-Motorist Substance Abuse      75458
Person ID                             0
Injury Severity                       0
Circumstance                      62690
Drivers License State              3920
Vehicle ID                            0


In [5]:
test.drop('Id', inplace=True, axis=1)
train.shape, test.shape

((51490, 42), (77235, 41))

In [6]:
def change_col_names(df):
    l = df.columns.tolist()
    for i in range(len(l)):
        l[i] = '_'.join(l[i].lower().split())
    df.columns = l
    return

def tolower(df):
    for col in df.columns:
        if df[col].dtype == 'O':
            df[col] = df[col].str.lower()

In [7]:
change_col_names(train)
change_col_names(test)
tolower(train)
tolower(test)
train.head()

,report_number,local_case_number,agency_name,acrs_report_type,crash_date/time,route_type,road_name,cross-street_type,cross-street_name,off-road_description,municipality,related_non-motorist,collision_type,weather,surface_condition,light,traffic_control,driver_substance_abuse,non-motorist_substance_abuse,person_id,injury_severity,circumstance,drivers_license_state,vehicle_id,vehicle_damage_extent,vehicle_first_impact_location,vehicle_second_impact_location,vehicle_body_type,vehicle_movement,vehicle_continuing_dir,vehicle_going_dir,speed_limit,driverless_vehicle,parked_vehicle,vehicle_year,vehicle_make,vehicle_model,equipment_problems,latitude,longitude,location,fault
0,mp060d,10196,montgomery county police,injury crash,06/05/2017 04:27:00 pm,maryland (state),old hundred rd,county,thurston rd,12305 shorefield rd,NaN,NaN,single vehicle,clear,wet,daylight,no controls,none detected,NaN,466d8b-ac8c7,suspected minor injury,"n/a, wet",md,7c2fec-727f4,destroyed,twelve oclock,twelve oclock,pickup truck,moving constant speed,south,unknown,40,no,no,2005,gmc,truck,no misuse,39.263378,-77.344203,3.6383 7.4233,1
1,mp000x,20080,montgomery county police,injury crash,02/22/2020 10:00:00 am,county,janet rd,county,flack st,NaN,NaN,NaN,straight movement angle,clear,dry,daylight,stop sign,none detected,NaN,4028e4-c5384,no apparent injury,"backup due to regular congestion, sleet, hail,...",md,67644f-64ef8,disabling,twelve oclock,twelve oclock,passenger car,accelerating,south,south,25,no,yes,2018,ford,fiesta,no misuse,39.068913,-77.063227,3.0862 7.324,1
2,mp070n,10103,montgomery county police,property damage crash,06/11/2017 08:21:00 am,maryland (state),colesville rd,maryland (state),struc #15082,NaN,NaN,NaN,same direction right turn,clear,dry,unknown,other,NaN,NaN,889dbd-d57ff,no apparent injury,NaN,dc,30f7a9-92c98,superficial,six oclock,six oclock,transit bus,parking,west,west,35,no,no,2016,new flyer,tbu,no misuse,38.993578,-77.032718,3.9583 7.3783,0
3,mp130y,10208,montgomery county police,injury crash,05/10/2019 07:30:00 am,county,parkland dr,county,frankfort dr,NaN,NaN,NaN,other,cloudy,dry,daylight,no controls,none detected,NaN,77155d-96aa7,possible injury,NaN,md,0f0c58-8e98f,functional,twelve oclock,two oclock,passenger car,moving constant speed,south,south,25,no,no,2001,chev,suburban,other,39.083480,-77.086933,"3.84,-70633",1
4,mp770y,1091,montgomery county police,injury crash,08/02/2016 05:00:00 pm,maryland (state),old georgetown rd,county,mckinley st,NaN,NaN,NaN,same dir rear end,clear,dry,daylight,traffic signal,none detected,medication present,5e180b-42b0c,suspected minor injury,NaN,md,4397c0-aee83,disabling,six oclock,six oclock,passenger car,slowing or stopping,south,south,40,no,no,2013,kia,rio,no misuse,38.996942,-77.109168,3.9917 7.0183,0


In [8]:
test.head()

,report_number,local_case_number,agency_name,acrs_report_type,crash_date/time,route_type,road_name,cross-street_type,cross-street_name,off-road_description,municipality,related_non-motorist,collision_type,weather,surface_condition,light,traffic_control,driver_substance_abuse,non-motorist_substance_abuse,person_id,injury_severity,circumstance,drivers_license_state,vehicle_id,vehicle_damage_extent,vehicle_first_impact_location,vehicle_second_impact_location,vehicle_body_type,vehicle_movement,vehicle_continuing_dir,vehicle_going_dir,speed_limit,driverless_vehicle,parked_vehicle,vehicle_year,vehicle_make,vehicle_model,equipment_problems,latitude,longitude,location
0,mp810m,NaN,montgomery,property damage crash,12/16/2015 05:42:00 pm,county,whittier blvd,maryland (state),wilson la,NaN,NaN,NaN,other,clear,dry,dark lights on,traffic signal,alcohol contributed,NaN,f05a35-c3c98,no apparent injury,NaN,md,da635f-c3055,functional,eleven oclock,eleven oclock,passenger car,making right turn,north,north,25,no,no,2004,lexus,rx330,no misuse,38.988440,-77.127668,"3.84,-71763"
1,mp2001,NaN,montgomery county police,property damage crash,05/01/2016 07:25:00 pm,county,railroad st,municipality,e diamond ave,NaN,gaithersburg,NaN,same dir rear end,raining,wet,daylight,traffic signal,none detected,NaN,79c1d8-6a412,no apparent injury,"rain, snow, wet",md,dbc8e9-fc340,superficial,six oclock,six oclock,passenger car,stopped in traffic lane,south,west,25,no,no,2001,mitsubishi,mirage,no misuse,39.094075,-77.205783,3.905 7.0733
2,mp6404,10125,montgomery county police,property damage crash,03/16/2018 02:30:00 pm,maryland (state),georgia ave,county,arcola ave,NaN,NaN,NaN,same direction sideswipe,NaN,dry,daylight,traffic signal,none detected,NaN,30ec44-418a7,no apparent injury,NaN,md,32da16-845e9,no damage,two oclock,two oclock,ambulance/emergency,moving constant speed,north,north,35,no,no,2016,frieghtliner,ambulance,no misuse,39.047180,-77.052688,"3.41,-70283"
3,mp0405,10270,montgomery county police,property damage crash,05/25/2018 05:00:00 pm,NaN,NaN,NaN,NaN,"19600 frederick rd, pawn shop parking lot",NaN,NaN,same direction sideswipe,clear,NaN,daylight,no controls,NaN,NaN,c7fa23-4267e,no apparent injury,NaN,md,4dc569-ab8ea,disabling,twelve oclock,twelve oclock,passenger car,accelerating,west,west,25,no,no,2004,ford,cn,NaN,39.178007,-77.238475,3.7067 7.345
4,mp090d,10374,montgomery county police,property damage crash,09/23/2017 05:40:00 pm,interstate (state),eisenhower memorial hwy,maryland (state),montrose rd,NaN,NaN,NaN,other,clear,dry,daylight,no controls,unknown,NaN,b6023b-a34ff,no apparent injury,NaN,NaN,b3bd7b-487e4,superficial,nine oclock,nine oclock,"medium/heavy trucks 3 axles (over 10,000lbs (4...",changing lanes,north,north,55,no,no,2008,util,tl,unknown,39.056134,-77.154043,3.5143 7.5022


In [11]:
train_dls = train.drivers_license_state.unique().tolist()
test_dls = test.drivers_license_state.unique().tolist()
other, othert = [], []

for i in train_dls:
    if i not in test_dls:
        other.append(i)
        
for i in test_dls:
    if i not in train_dls:
        othert.append(i)

In [12]:
len(train_dls), len(test_dls), other, othert

(66, 74, ['gu'], ['um', 'it', 'ns', 'mp', 'nt', 'sd', 'nf', 'fm', 'sk'])

In [13]:
train.drivers_license_state = train.drivers_license_state.replace(dict.fromkeys(other, 'other'))
test.drivers_license_state = test.drivers_license_state.replace(dict.fromkeys(othert, 'other'))

In [14]:
train.drivers_license_state.unique().shape, test.drivers_license_state.unique().shape

((66,), (66,))

In [12]:
for col in train.columns:
    print('{} --> {}'.format(col, train[col].unique()), end='\n\n')

report_number --> ['mp060d' 'mp000x' 'mp070n' ... 'mp5708' 'mp710m' 'd5702']

local_case_number --> ['10196' '20080' '10103' ... '1505' '1562' '1517']

agency_name --> ['montgomery county police' 'rockville police departme' 'montgomery'
 'gaithersburg' 'takoma' 'rockville' 'gaithersburg police depar'
 'maryland-national capital' 'takoma park police depart' 'mcpark']

acrs_report_type --> ['injury crash' 'property damage crash' 'fatal crash']

crash_date/time --> ['06/05/2017 04:27:00 pm' '02/22/2020 10:00:00 am'
 '06/11/2017 08:21:00 am' ... '10/04/2016 03:17:00 pm'
 '07/05/2016 09:52:00 am' '05/16/2017 05:37:00 am']

route_type --> ['maryland (state)' 'county' nan 'us (state)' 'ramp' 'interstate (state)'
 'municipality' 'government' 'unknown' 'other public roadway'
 'service road']

road_name --> ['old hundred rd' 'janet rd' 'colesville rd' ... 'twin ridge ct'
 'whitehaven rd' 'honesty way']

cross-street_type --> ['county' 'maryland (state)' 'municipality' 'other public roadway' nan


In [15]:
print(train.latitude.min(), test.latitude.min())
print(train.latitude.max(), test.latitude.max())
print(train.longitude.min(), test.longitude.min())
print(train.longitude.max(), test.longitude.max())

38.00812 37.72
39.98974718 39.990414
-79.486 -79.486
-75.97595215 -75.52770787


In [16]:
train['crash_datetime'] = pd.to_datetime(train['crash_date/time'])
train['crash_year'] = train.crash_datetime.dt.year
train['crash_month'] = train.crash_datetime.dt.month
train['crash_week'] = train.crash_datetime.dt.week
train['crash_weekday'] = train.crash_datetime.dt.weekday
train['crash_day'] = train.crash_datetime.dt.day
train['crash_hour'] = train.crash_datetime.dt.hour
train.agency_name = train.agency_name.apply(lambda x: x.split()[0])
train.latitude = train.latitude.round(3)
train.longitude = train.longitude.round(3)


test['crash_datetime'] = pd.to_datetime(test['crash_date/time'])
test['crash_year'] = test.crash_datetime.dt.year
test['crash_month'] = test.crash_datetime.dt.month
test['crash_week'] = test.crash_datetime.dt.week
test['crash_weekday'] = test.crash_datetime.dt.weekday
test['crash_day'] = test.crash_datetime.dt.day
test['crash_hour'] = test.crash_datetime.dt.hour
test.agency_name = test.agency_name.apply(lambda x: x.split()[0])
test.latitude = test.latitude.round(3)
test.longitude = test.longitude.round(3)

<ipython-input-16-90c025eb3051>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  train['crash_week'] = train.crash_datetime.dt.week
<ipython-input-16-90c025eb3051>:16: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  test['crash_week'] = test.crash_datetime.dt.week


In [17]:
train.agency_name.unique()

array(['montgomery', 'rockville', 'gaithersburg', 'takoma',
       'maryland-national', 'mcpark'], dtype=object)

In [18]:
test.agency_name.unique()

array(['montgomery', 'rockville', 'gaithersburg', 'maryland-national',
       'mcpark', 'takoma'], dtype=object)

In [19]:
to_drop = []
for col in train.columns:
    if train[col].isna().sum() >11000:
        to_drop.append((col))
to_drop

['off-road_description',
 'municipality',
 'related_non-motorist',
 'non-motorist_substance_abuse',
 'circumstance']

In [20]:
to_dropt = []
for col in test.columns:
    if test[col].isna().sum() >11000:
        to_dropt.append((col))
to_dropt

['local_case_number',
 'off-road_description',
 'municipality',
 'related_non-motorist',
 'traffic_control',
 'driver_substance_abuse',
 'non-motorist_substance_abuse',
 'circumstance',
 'equipment_problems']

In [21]:
to_dropt += ['person_id', 'vehicle_id', 'vehicle_make', 'location', 'crash_datetime', 'crash_date/time']

In [22]:
train.drop(to_dropt, axis=1, inplace=True)
test.drop(to_dropt, axis=1, inplace=True)
train.shape, test.shape

((51490, 34), (77235, 33))

In [23]:
train.head()

,report_number,agency_name,acrs_report_type,route_type,road_name,cross-street_type,cross-street_name,collision_type,weather,surface_condition,light,injury_severity,drivers_license_state,vehicle_damage_extent,vehicle_first_impact_location,vehicle_second_impact_location,vehicle_body_type,vehicle_movement,vehicle_continuing_dir,vehicle_going_dir,speed_limit,driverless_vehicle,parked_vehicle,vehicle_year,vehicle_model,latitude,longitude,fault,crash_year,crash_month,crash_week,crash_weekday,crash_day,crash_hour
0,mp060d,montgomery,injury crash,maryland (state),old hundred rd,county,thurston rd,single vehicle,clear,wet,daylight,suspected minor injury,md,destroyed,twelve oclock,twelve oclock,pickup truck,moving constant speed,south,unknown,40,no,no,2005,truck,39.263,-77.344,1,2017,6,23,0,5,16
1,mp000x,montgomery,injury crash,county,janet rd,county,flack st,straight movement angle,clear,dry,daylight,no apparent injury,md,disabling,twelve oclock,twelve oclock,passenger car,accelerating,south,south,25,no,yes,2018,fiesta,39.069,-77.063,1,2020,2,8,5,22,10
2,mp070n,montgomery,property damage crash,maryland (state),colesville rd,maryland (state),struc #15082,same direction right turn,clear,dry,unknown,no apparent injury,dc,superficial,six oclock,six oclock,transit bus,parking,west,west,35,no,no,2016,tbu,38.994,-77.033,0,2017,6,23,6,11,8
3,mp130y,montgomery,injury crash,county,parkland dr,county,frankfort dr,other,cloudy,dry,daylight,possible injury,md,functional,twelve oclock,two oclock,passenger car,moving constant speed,south,south,25,no,no,2001,suburban,39.083,-77.087,1,2019,5,19,4,10,7
4,mp770y,montgomery,injury crash,maryland (state),old georgetown rd,county,mckinley st,same dir rear end,clear,dry,daylight,suspected minor injury,md,disabling,six oclock,six oclock,passenger car,slowing or stopping,south,south,40,no,no,2013,rio,38.997,-77.109,0,2016,8,31,1,2,17


In [24]:
test.head()

,report_number,agency_name,acrs_report_type,route_type,road_name,cross-street_type,cross-street_name,collision_type,weather,surface_condition,light,injury_severity,drivers_license_state,vehicle_damage_extent,vehicle_first_impact_location,vehicle_second_impact_location,vehicle_body_type,vehicle_movement,vehicle_continuing_dir,vehicle_going_dir,speed_limit,driverless_vehicle,parked_vehicle,vehicle_year,vehicle_model,latitude,longitude,crash_year,crash_month,crash_week,crash_weekday,crash_day,crash_hour
0,mp810m,montgomery,property damage crash,county,whittier blvd,maryland (state),wilson la,other,clear,dry,dark lights on,no apparent injury,md,functional,eleven oclock,eleven oclock,passenger car,making right turn,north,north,25,no,no,2004,rx330,38.988,-77.128,2015,12,51,2,16,17
1,mp2001,montgomery,property damage crash,county,railroad st,municipality,e diamond ave,same dir rear end,raining,wet,daylight,no apparent injury,md,superficial,six oclock,six oclock,passenger car,stopped in traffic lane,south,west,25,no,no,2001,mirage,39.094,-77.206,2016,5,17,6,1,19
2,mp6404,montgomery,property damage crash,maryland (state),georgia ave,county,arcola ave,same direction sideswipe,NaN,dry,daylight,no apparent injury,md,no damage,two oclock,two oclock,ambulance/emergency,moving constant speed,north,north,35,no,no,2016,ambulance,39.047,-77.053,2018,3,11,4,16,14
3,mp0405,montgomery,property damage crash,NaN,NaN,NaN,NaN,same direction sideswipe,clear,NaN,daylight,no apparent injury,md,disabling,twelve oclock,twelve oclock,passenger car,accelerating,west,west,25,no,no,2004,cn,39.178,-77.238,2018,5,21,4,25,17
4,mp090d,montgomery,property damage crash,interstate (state),eisenhower memorial hwy,maryland (state),montrose rd,other,clear,dry,daylight,no apparent injury,NaN,superficial,nine oclock,nine oclock,"medium/heavy trucks 3 axles (over 10,000lbs (4...",changing lanes,north,north,55,no,no,2008,tl,39.056,-77.154,2017,9,38,5,23,17


In [25]:
more_drop = []
for col in train.columns:
    if len(train[col].unique())>100 and train[col].dtype=='O':
        more_drop.append(col)
    print(col, len(train[col].unique()))

report_number 3256
agency_name 6
acrs_report_type 3
route_type 11
road_name 2227
cross-street_type 11
cross-street_name 4526
collision_type 19
weather 13
surface_condition 12
light 9
injury_severity 5
drivers_license_state 66
vehicle_damage_extent 8
vehicle_first_impact_location 17
vehicle_second_impact_location 17
vehicle_body_type 31
vehicle_movement 23
vehicle_continuing_dir 6
vehicle_going_dir 6
speed_limit 15
driverless_vehicle 2
parked_vehicle 2
vehicle_year 83
vehicle_model 3129
latitude 427
longitude 567
fault 2
crash_year 7
crash_month 12
crash_week 53
crash_weekday 7
crash_day 31
crash_hour 24


In [26]:
more_drop

['report_number', 'road_name', 'cross-street_name', 'vehicle_model']

In [27]:
train.drop(more_drop, axis=1, inplace=True)
test.drop(more_drop, axis=1, inplace=True)
train.shape, test.shape

((51490, 30), (77235, 29))

In [28]:
train.isna().sum()

agency_name                          0
acrs_report_type                     0
route_type                        4968
cross-street_type                 5019
collision_type                     299
weather                           4029
surface_condition                 5879
light                              704
injury_severity                      0
drivers_license_state             2528
vehicle_damage_extent              397
vehicle_first_impact_location      222
vehicle_second_impact_location     234
vehicle_body_type                  785
vehicle_movement                   198
vehicle_continuing_dir            1129
vehicle_going_dir                 1113
speed_limit                          0
driverless_vehicle                   0
parked_vehicle                       0
vehicle_year                         0
latitude                             0
longitude                            0
fault                                0
crash_year                           0
crash_month              

In [29]:
test.isna().sum()

agency_name                          0
acrs_report_type                     0
route_type                        7467
cross-street_type                 7480
collision_type                     245
weather                           5996
surface_condition                 9053
light                              620
injury_severity                      0
drivers_license_state             3920
vehicle_damage_extent              136
vehicle_first_impact_location       81
vehicle_second_impact_location     139
vehicle_body_type                 1107
vehicle_movement                   153
vehicle_continuing_dir            1201
vehicle_going_dir                 1201
speed_limit                          0
driverless_vehicle                   0
parked_vehicle                       0
vehicle_year                         0
latitude                             0
longitude                            0
crash_year                           0
crash_month                          0
crash_week               

In [30]:
for col in train.columns:
    if train[col].dtype=='O':
        train[col] = train[col].fillna(train[col].mode()[0])

for col in test.columns:
    if test[col].dtype=='O':
        test[col] = test[col].fillna(test[col].mode()[0])

In [31]:
train.isna().sum()

agency_name                       0
acrs_report_type                  0
route_type                        0
cross-street_type                 0
collision_type                    0
weather                           0
surface_condition                 0
light                             0
injury_severity                   0
drivers_license_state             0
vehicle_damage_extent             0
vehicle_first_impact_location     0
vehicle_second_impact_location    0
vehicle_body_type                 0
vehicle_movement                  0
vehicle_continuing_dir            0
vehicle_going_dir                 0
speed_limit                       0
driverless_vehicle                0
parked_vehicle                    0
vehicle_year                      0
latitude                          0
longitude                         0
fault                             0
crash_year                        0
crash_month                       0
crash_week                        0
crash_weekday               

In [32]:
test.isna().sum()

agency_name                       0
acrs_report_type                  0
route_type                        0
cross-street_type                 0
collision_type                    0
weather                           0
surface_condition                 0
light                             0
injury_severity                   0
drivers_license_state             0
vehicle_damage_extent             0
vehicle_first_impact_location     0
vehicle_second_impact_location    0
vehicle_body_type                 0
vehicle_movement                  0
vehicle_continuing_dir            0
vehicle_going_dir                 0
speed_limit                       0
driverless_vehicle                0
parked_vehicle                    0
vehicle_year                      0
latitude                          0
longitude                         0
crash_year                        0
crash_month                       0
crash_week                        0
crash_weekday                     0
crash_day                   

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   agency_name                     51490 non-null  object 
 1   acrs_report_type                51490 non-null  object 
 2   route_type                      51490 non-null  object 
 3   cross-street_type               51490 non-null  object 
 4   collision_type                  51490 non-null  object 
 5   weather                         51490 non-null  object 
 6   surface_condition               51490 non-null  object 
 7   light                           51490 non-null  object 
 8   injury_severity                 51490 non-null  object 
 9   drivers_license_state           51490 non-null  object 
 10  vehicle_damage_extent           51490 non-null  object 
 11  vehicle_first_impact_location   51490 non-null  object 
 12  vehicle_second_impact_location  

In [34]:
_,_, obj_col = train.columns.to_series().groupby(train.dtypes).groups.items()
obj_col = obj_col[1].tolist()
obj_col

['agency_name',
 'acrs_report_type',
 'route_type',
 'cross-street_type',
 'collision_type',
 'weather',
 'surface_condition',
 'light',
 'injury_severity',
 'drivers_license_state',
 'vehicle_damage_extent',
 'vehicle_first_impact_location',
 'vehicle_second_impact_location',
 'vehicle_body_type',
 'vehicle_movement',
 'vehicle_continuing_dir',
 'vehicle_going_dir',
 'driverless_vehicle',
 'parked_vehicle']

In [35]:
train_data = pd.get_dummies(train, columns=obj_col, drop_first = True)

In [36]:
_,_, obj_colt = test.columns.to_series().groupby(test.dtypes).groups.items()
obj_colt = obj_colt[1].tolist()
obj_colt

['agency_name',
 'acrs_report_type',
 'route_type',
 'cross-street_type',
 'collision_type',
 'weather',
 'surface_condition',
 'light',
 'injury_severity',
 'drivers_license_state',
 'vehicle_damage_extent',
 'vehicle_first_impact_location',
 'vehicle_second_impact_location',
 'vehicle_body_type',
 'vehicle_movement',
 'vehicle_continuing_dir',
 'vehicle_going_dir',
 'driverless_vehicle',
 'parked_vehicle']

In [37]:
test_data = pd.get_dummies(test, columns=obj_colt, drop_first = True)

In [38]:
train_data.shape, test_data.shape

((51490, 245), (77235, 244))

In [39]:
X_train, y_train = train_data.drop('fault', axis=1), train_data.fault

In [40]:
rfr = RandomForestClassifier(n_jobs=6)

In [52]:
params = {
    'max_depth': [10, 50, 100],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [1000, 1500]
}
gs = GridSearchCV(estimator = RandomForestClassifier(max_features='sqrt', random_state=999), param_grid=params, scoring='accuracy', n_jobs=6, cv=5)
gs = gs.fit(X_train, y_train)
best_accuracy = gs.best_score_
best_params = gs.best_params_

/home/kgautam/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [53]:
best_params, best_accuracy

({'n_estimators': 1000}, 0.8438920178675471)

In [54]:
gs.cv_results_

{'mean_fit_time': array([ 64.81684728, 103.06577358, 119.03738737]),
 'std_fit_time': array([0.23550946, 1.50509692, 3.71773864]),
 'mean_score_time': array([1.92983894, 3.21039352, 3.55245218]),
 'std_score_time': array([0.0120019 , 0.03937952, 0.37460395]),
 'param_n_estimators': masked_array(data=[500, 800, 1000],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 500},
  {'n_estimators': 800},
  {'n_estimators': 1000}],
 'split0_test_score': array([0.84521266, 0.84501845, 0.84453292]),
 'split1_test_score': array([0.84249369, 0.84346475, 0.84307633]),
 'split2_test_score': array([0.84229948, 0.84327054, 0.84395028]),
 'split3_test_score': array([0.8457953 , 0.84647504, 0.84647504]),
 'split4_test_score': array([0.84152263, 0.84045446, 0.84142552]),
 'mean_test_score': array([0.84346475, 0.84373665, 0.84389202]),
 'std_test_score': array([0.00170642, 0.00201026, 0.00166344]),
 'rank_test_score': array([3, 2, 1],